# Desafio 2

<hr>

#### Contexto:

Nossa equipe de inteligência de negócios precisa analisar a receita de todas as lojas de uma
rede de restaurantes.

Essas informações podem ser obtidas por meio de 5 endpoints de API.


| **Method** | **API/Endpoint**                    | **Payload**              |
|------------|-------------------------------------|--------------------------|
| POST       | /bi/getFiscalInvoice               | - busDt: string(date)    |
|            |                                     | - storeId: string        |
| POST       | /res/getGuestChecks                | - busDt: string(date)    |
|            |                                     | - storeId: string        |
| POST       | /org/getChargeBack                 | - busDt: string(date)    |
|            |                                     | - storeId: string        |
| POST       | /trans/getTransactions             | - busDt: string(date)    |
|            |                                     | - storeId: string        |
| POST       | /inv/getCashManagementDetails      | - busDt: string(date)    |
|            |                                     | - storeId: string        |


###### *storeId corresponde ao identificador da loja.

###### *busDt corresponde à data de operação.



**Tarefa**: Esta é uma continuação do primeiro desafio. Você deve armazenar os dados das
respostas das APIs (JSON) no nosso data lake

## 1. Por que armazenar as respostas das APIs?

Armazenar as respostas das APIs permite preservar informações temporais, como vendas diárias e dados fiscais, fundamentais para análises históricas e decisões estratégicas. Sem esse armazenamento, a equipe de inteligência de negócios teria acesso apenas a dados recentes, limitando a identificação de padrões e tendências. Além disso, a centralização das informações em um data lake integra os dados das diferentes APIs, facilitando análises cruzadas, como combinar vendas com cobranças adicionais, eliminando a fragmentação e garantindo consistência nas operações. Essa prática também reduz a carga nos sistemas de produção ao evitar requisições contínuas, permitindo que as análises sejam feitas diretamente nos dados armazenados. O modelo é escalável, acompanhando o crescimento da rede e suportando novas demandas, como previsões de receita.

<hr>

## 2. Como você armazenaria os dados? Crie uma estrutura de pastas capaz de armazenar as respostas da API. Ela deve permitir manipulaçõe, verificações, buscas e pesquisas rápidas.

In [ ]:
/data_lake/
    /api_data/
        /{endpoint}/
            /year={YYYY}/
                /month={MM}/
                    /day={DD}/
                        /storeId={storeId}/
                            {endpoint}_{busDt}_{storeId}.json


#### Explicação da Estrutura

**Nível 1 - Diretório Raiz (/data_lake/api_data/):**

Todos os dados das APIs ficam centralizados em um único local, garantindo que diferentes fontes sejam armazenadas de maneira consistente.

**Nível 2 - Diretório por Endpoint (/{endpoint}/):**

Cada API tem seu próprio diretório (ex.: /bi/getFiscalInvoice/, /res/getGuestChecks/), facilitando a segmentação dos dados por fonte.
Isso permite acessar rapidamente os dados de um endpoint específico sem interferir nos outros.

**Nível 3 - Particionamento Temporal (/year={YYYY}/month={MM}/day={DD}/):**

Os dados são particionados por ano, mês e dia, alinhados à propriedade busDt do payload.
Esse particionamento facilita buscas rápidas em consultas temporais, como "dados de janeiro de 2024".

**Nível 4 - Diretório por Loja (/storeId={storeId}/):**

Dentro de cada dia, os dados são armazenados por storeId, permitindo consultas específicas para uma loja individual.

**Arquivos Nomeados Descritivamente:**

Nome do arquivo: {endpoint}_{busDt}_{storeId}.json
Exemplo: getGuestChecks_2024-01-01_99CB.json
Isso ajuda a identificar rapidamente o conteúdo de um arquivo sem precisar abrir ou processá-lo.

<hr>

## 3. Considere que a resposta do endpoint getGuestChecks foi alterada, por exemplo, guestChecks.taxes foi renomeado para guestChecks.taxation. O que isso implicaria?


A mudança de guestChecks.taxes para guestChecks.taxation impacta diretamente os pipelines de ETL, o armazenamento no data lake e as análises existentes. Sem ajustes, os processos que dependem de taxes falharão ao encontrar o novo campo. Para resolver, é necessário implementar uma transformação no pipeline que normalize o nome, como no exemplo:

In [ ]:
def normalize_schema(data):
    if "taxation" in data["guestChecks"]:
        data["guestChecks"]["taxes"] = data["guestChecks"].pop("taxation")
    return data

Esse é um exemplo simples que pode garantir a integridade dos dados em uma pipeline. Além disso, consultas e relatórios devem ser adaptados para lidar com ambos os nomes durante a transição, usando lógica como:

In [ ]:
def get_taxes(data):
    return data["guestChecks"].get("taxes") or data["guestChecks"].get("taxation")

Para evitar problemas futuros, pode ser adicionado uma versão de schema nos dados armazenados e comunicar claramente a mudança aos interessados. Com esses ajustes, a operação segue sem interrupções e mantém a integridade dos dados.